# Image Generator

In [ ]:
#| default_exp image_generator

In [ ]:
#| eval:false
from Cloud2DImageConverter import spherical_projection as sp
from Cloud2DImageConverter import data
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import shutil
import time
import os

Carregando os dados

In [ ]:
#| eval:false
point_cloud = data.load_data("../point_clouds/semantic_kitti")

Gerando as projeções* https://github.com/alunos-pfc/Spherical-Projection-SemanticKITTI


In [ ]:
#| eval:false
projection_dict = {
    "reflectance": [],
    "label": []
}

for points in point_cloud:
    unlabaled, labaled = sp.spherical_projection(points)
    labaled = sp.colored_matrix_with_label(labaled)
    projection_dict["reflectance"].append(unlabaled)
    projection_dict["label"].append(labaled)

In [ ]:
#| eval:false
def image_generator(projection_dict, output_folder, rows=64, cols=1024, delete=True):
    start_time = time.time() 
    results_folder =  output_folder+"results"

    # Se a pasta já existir, apague-a 
    if os.path.exists(results_folder) and delete:
        shutil.rmtree(results_folder)
        
    os.makedirs(results_folder)
    os.makedirs(results_folder+"/reflectance")
    os.makedirs(results_folder+"/label")

    for zip_dict in zip(projection_dict["reflectance"], projection_dict["label"]):
        for i, value in enumerate(projection_dict.keys()): 
            # Aplica o cmap manualmente 
            if value == "reflectance":
                img = zip_dict[i]
                cmap = plt.get_cmap("jet")
                img = (cmap(img)*255).astype(np.uint8)
                img = Image.fromarray(img)
            else:
                img = Image.fromarray(zip_dict[i])
                
            # dict["val"] é o valor de reflectance ou label
            folder_type = "/" + value

            # Caminho final para salvar
            final_path = os.path.join(results_folder+folder_type)

            # Define o nome do arquivo, tais como 000001, 000002...
            file_number = len(os.listdir(final_path)) + 1
            file_name = f"{final_path}/{file_number:06d}.png"

            img.save(file_name)

    # Registra o tempo de fim da execução
    end_time = time.time() 
    elapsed_time = end_time - start_time
    print(f"Tempo de execução: {elapsed_time} segundos")

image_generator(projection_dict, output_folder="../point_clouds/")

Tempo de execução: 5.677505731582642 segundos
